In [39]:
import numpy as np
import pandas as pd
class dataParser():
    def __init__(self,day,timerange,dataFileNames=['incidents2009.csv']):
        #Emergency Medical Response,FS09000001,San Diego,Stabbing/Gunshot (L1),2800 BROADWAY,28TH ST/29TH ST,SAN DIEGO,92102,1/1/09 0:05:49,1/1/09 0:08:08,0:02:19
        self.data = []
		for fileName in dataFileNames:
			self.dataFile = open(fileName,'rU')
			first = 1
			for line in self.dataFile:
				if first == 1:
					first = 0
					continue
				splitList = self.splitComma(line.strip())
				dateTime = self.getDateTime(splitList[8])
				splitList = self.fixLatLong(splitList)
				if len(splitList) != 11:
					print splitList
					raise Exception('Data is not proper number of fields')
				if dateTime >= self.start and dateTime <= self.end:
					self.data.append([dateTime, splitList])
			self.dataLength = len(self.data)

	def splitComma(self,myStr):
		#necessary because data has form _one,_two,"three, three continued", four
		returnList = []
		previous = 0
		inquotes = 0
		for i in xrange(0,len(myStr)):
			if myStr[i] == '"':
				if inquotes == 1:
					inquotes = 0
				else:
					inquotes = 1
			if myStr[i] == ",":
				if inquotes == 0:
					returnList.append(myStr[previous:i])
					previous = i+1
		returnList.append(myStr[previous:len(myStr)])
		return returnList


	def fixLatLong(self,myList):
		#sorry about this, latLong was formatted as 'lat-long', needs to be 'lat_-long'
		myList[4] = myList[4].split('-')[0]+'_-'+myList[4].split('-')[1]
		return myList


	def getDateTime(self,timeString):
		#year, month, day, hour*24*60 + minute*60 + second
		#example:   11/23/09 20:42:52
		myList = timeString.strip().split()[0].split('/')
		timeList = timeString.strip().split()[1].split(':')
		myDateTime = datetime.datetime(int(myList[2]),int(myList[0]),int(myList[1]),int(timeList[0]),int(timeList[1]),int(timeList[2]))
		return myDateTime


	def dispenseData(self):
		startTime = self.data[0][0]
		currentTime = startTime
		endTime = self.data[-1][0]
		timeStep = datetime.timedelta(0,self.timeStep)
		currentElement = 0
		currentTime = currentTime - timeStep
		dataMapper = []
		timeStepNumber = 0 
		while currentTime < endTime:
			while self.data[currentElement][0] < currentTime:
				currentElement+=1
				dataMapper.append(timeStepNumber)
				#maps each piece of data to a time step number
				if currentElement >= self.dataLength:
					break
			currentTime = currentTime +timeStep
			timeStepNumber += 1

		modelInstance = dataModel(timeStepNumber)
		timeStepNumber = 0
		i=0

		while i < len(dataMapper):
			passList = []
			while dataMapper[i] <= timeStepNumber:
				passList.append(self.data[i][1])
				i+=1
				if i == len(dataMapper):
					break
			print "Timestep: ", timeStepNumber
			modelInstance.receiveNextData(passList, timeStepNumber)
			timeStepNumber+=1

In [40]:
hourlyResults = []
class dataModel():
    def __init__(self,timeStepsIncoming):
		self.expectedTimeSteps = timeStepsIncoming #Added this to change the exploration prob halfway through
		self.setup = 0
		self.numTrucks = 3  # must be >= 1
		self.stepSize = 1
		self.hourlyResults = []
		self.ongoingIncidents = {}
		self.allIncidents = {}
		self.resolvedIncidents = {}
		self.resolvedtrainortest = {}
		self.gridHorizontalGranularity = 10 # must be > 1
		self.gridVerticleGranularity = 10 # must be > 1
		self.grid = self.getGrid()
		self.totalError = 0
		self.currentTime = -1
		self.trainingOrTesting = 'training'
        
    def getGrid(self):
		#grid[0][0] is the northwest corner of sanDiego and grid[n][m] is the southeast.
		#This means that the latitude  at grid[0][0] is greater than the latitude  at grid[n][m]
		#and 			 the longitude at grid[0][0] is less    than the longitude at grid[n][m]
		corners = [(32.981625, -117.270982), (32.692295, -117.009370)]

		vertSteps = self.gridVerticleGranularity
		horizontalSteps = self.gridHorizontalGranularity
		deltaLat = abs(corners[0][0]-corners[1][0]) 
		deltaLong = abs(corners[0][1]-corners[1][1]) 
		grid = []
		for r in xrange(0,self.gridVerticleGranularity):
			tempRow = []
			for c in xrange(0,self.gridHorizontalGranularity):
				curLat = corners[0][0] - r*(deltaLat/(self.gridVerticleGranularity-1))
				curLong = corners[0][1] + c*(deltaLong/(self.gridHorizontalGranularity-1))
				tempRow.append((curLat,curLong))
			grid.append(tempRow)
		return grid
    
    def whereOnGrid(self,lat,longd):
		curRow = 0
		curCol = 0
		if lat < self.grid[len(self.grid)-1][curCol][0]:
			curRow = len(self.grid)-1
		else:
			while lat < self.grid[curRow][curCol][0]:
				curRow += 1
		if longd > self.grid[curRow][len(self.grid[curRow])-1][1]:
			curCol = len(self.grid[curRow])-1
		else:
			while longd > self.grid[curRow][curCol][1]:
				curCol += 1
		return (curRow,curCol)
    
    def updateModel(self, listOfData, timestep):
        self.currentTime = timestep
        hResults = collections.defaultdict(int)
        for datapoint in listOfData:
            incident_key = datapoint[1]
            lati = float(datapoint[4].split('_')[0])
            lond = float(datapoint[4].split('_')[1])
            location = self.whereOnGrid(lati, lond)
            hResults[location] += 1
            self.ongoingIncidents[incident_key] = location
            self.allIncidents[incident_key] = (self.currentTime, location)
            self.resolvedtrainortest[incident_key] = self.trainingOrTesting
        hourlyResults.append((timestep, hResults))
    
    def receiveNextData(self, listOfData, timestep):
        self.updateModel(listOfData, timestep)

In [41]:
import datetime
import random
#import geocoder
import math
import sys
import copy
import collections
dd = dataDispenser(datetime.datetime(9,1,1),datetime.timedelta(5))
dd.dispenseData()

Timestep:  0
Timestep:  1
Timestep:  2
Timestep:  3
Timestep:  4
Timestep:  5
Timestep:  6
Timestep:  7
Timestep:  8
Timestep:  9
Timestep:  10
Timestep:  11
Timestep:  12
Timestep:  13
Timestep:  14
Timestep:  15
Timestep:  16
Timestep:  17
Timestep:  18
Timestep:  19
Timestep:  20
Timestep:  21
Timestep:  22
Timestep:  23
Timestep:  24
Timestep:  25
Timestep:  26
Timestep:  27
Timestep:  28
Timestep:  29
Timestep:  30
Timestep:  31
Timestep:  32
Timestep:  33
Timestep:  34
Timestep:  35
Timestep:  36
Timestep:  37
Timestep:  38
Timestep:  39
Timestep:  40
Timestep:  41
Timestep:  42
Timestep:  43
Timestep:  44
Timestep:  45
Timestep:  46
Timestep:  47
Timestep:  48
Timestep:  49
Timestep:  50
Timestep:  51
Timestep:  52
Timestep:  53
Timestep:  54
Timestep:  55
Timestep:  56
Timestep:  57
Timestep:  58
Timestep:  59
Timestep:  60
Timestep:  61
Timestep:  62
Timestep:  63
Timestep:  64
Timestep:  65
Timestep:  66
Timestep:  67
Timestep:  68
Timestep:  69
Timestep:  70
Timestep:  71
Ti

In [42]:
hourlyResults

[(0, defaultdict(int, {})),
 (1, defaultdict(int, {})),
 (2,
  defaultdict(int,
              {(0, 8): 1,
               (3, 1): 1,
               (5, 1): 1,
               (6, 1): 4,
               (6, 2): 1,
               (6, 7): 1,
               (6, 8): 1,
               (8, 2): 1,
               (8, 6): 1,
               (8, 7): 1,
               (9, 4): 8,
               (9, 5): 2,
               (9, 6): 1,
               (9, 7): 1,
               (9, 8): 3,
               (9, 9): 1})),
 (3,
  defaultdict(int,
              {(2, 1): 1,
               (2, 2): 1,
               (3, 1): 1,
               (4, 2): 1,
               (6, 1): 1,
               (6, 2): 1,
               (6, 5): 1,
               (6, 7): 1,
               (7, 5): 1,
               (8, 3): 1,
               (8, 5): 1,
               (8, 6): 1,
               (8, 7): 3,
               (9, 4): 5,
               (9, 5): 1,
               (9, 6): 1,
               (9, 7): 1})),
 (4,
  defaultdict(int,
        

In [3]:
import pandas as pd
d = pd.read_csv('incidents2009.csv')

ImportError: this version of pandas is incompatible with numpy < 1.7.0
your numpy version is 1.6.2.
Please upgrade numpy to >= 1.7.0 to use this pandas version